In [1]:
import pandas as pd
import numpy as np

In [245]:
data_path = './data/'
df_train = pd.read_csv(data_path + 'train.csv')
df_test = pd.read_csv(data_path + 'test.csv')

train_Y = df_train['Survived']
df_train = df_train.drop(['Survived'] , axis=1)

df = pd.concat([df_train,df_test])
print(df.shape)
df.head()

(1309, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [246]:
# Pcalss 無遺失值
print(np.where(pd.isnull(df[['Pclass']])))

(array([], dtype=int64), array([], dtype=int64))


In [247]:
# Name 無遺失值
print(np.where(pd.isnull(df[['Name']])))

(array([], dtype=int64), array([], dtype=int64))


In [248]:
# 使用title作為feature
df['title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
print(df['title'].unique())
df['title'].head()

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer' 'Dona']


0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: title, dtype: object

In [249]:
# Ticket 無遺失值
print(np.where(pd.isnull(df[['Ticket']])))
# df.sort_values(by=['Ticket'])

(array([], dtype=int64), array([], dtype=int64))


In [250]:
# Sex 無遺失值
print(np.where(pd.isnull(df[['Sex']])))

(array([], dtype=int64), array([], dtype=int64))


In [251]:
# Sibsp, Parch無遺失值
print(np.where(pd.isnull(df[['SibSp']])))
print(np.where(pd.isnull(df[['Parch']])))

df['Family'] = df['SibSp'] + df['Parch']

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [252]:
# Age 有遺失值須進行補值
print(np.where(pd.isnull(df[['Age']])))

(array([   5,   17,   19,   26,   28,   29,   31,   32,   36,   42,   45,
         46,   47,   48,   55,   64,   65,   76,   77,   82,   87,   95,
        101,  107,  109,  121,  126,  128,  140,  154,  158,  159,  166,
        168,  176,  180,  181,  185,  186,  196,  198,  201,  214,  223,
        229,  235,  240,  241,  250,  256,  260,  264,  270,  274,  277,
        284,  295,  298,  300,  301,  303,  304,  306,  324,  330,  334,
        335,  347,  351,  354,  358,  359,  364,  367,  368,  375,  384,
        388,  409,  410,  411,  413,  415,  420,  425,  428,  431,  444,
        451,  454,  457,  459,  464,  466,  468,  470,  475,  481,  485,
        490,  495,  497,  502,  507,  511,  517,  522,  524,  527,  531,
        533,  538,  547,  552,  557,  560,  563,  564,  568,  573,  578,
        584,  589,  593,  596,  598,  601,  602,  611,  612,  613,  629,
        633,  639,  643,  648,  650,  653,  656,  667,  669,  674,  680,
        692,  697,  709,  711,  718,  727,  732,  

In [253]:
# 用 Pcalss, title, Sex, Sibsp, Parch建立迴歸模型來預測 Age
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
df['Pclass'] = df['Pclass'].astype('category')
df_temp =  pd.get_dummies(df[['Pclass','title','Sex']])
df_temp['SibSp'] = df['SibSp']
df_temp['Parch'] = df['Parch']
df_temp['Age'] = df['Age']

df_temp_train = df_temp[df_temp[['Age']].isnull().values==False]
df_temp_value = df_temp_train['Age']
df_temp_train = df_temp_train.drop(['Age'], axis=1)

lasso = linear_model.Lasso(alpha=0.01)
lasso.fit(df_temp_train, df_temp_value)

df_temp_test = df_temp[df_temp[['Age']].isnull().values==True]
df_temp_test = df_temp_test.drop(['Age'], axis=1)
y_pred = list(lasso.predict(df_temp_test))
y_pred = [ round(elem, 0) for elem in y_pred ]

df.Age[df.Age.isna()] = y_pred

D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [254]:
print(np.where(pd.isnull(df[['Fare']])))

df['Fare'].fillna(df['Fare'].median(), inplace=True)

(array([1043], dtype=int64), array([0], dtype=int64))


In [255]:
# 針對Fare做標準化
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df['Fare_stand'] = StandardScaler().fit_transform(df[['Fare']])
df['Fare_minmax'] = MinMaxScaler().fit_transform(df[['Fare']])

In [256]:
# 補缺失的 Embarked
print(np.where(pd.isnull(df[['Embarked']])))
df.Embarked[df.Embarked.isna()] = df['Embarked'].mode()[0]

(array([ 61, 829], dtype=int64), array([0, 0], dtype=int64))


D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [257]:
#  補Cabin，並只取第一個字母
df['Cabin'].fillna("-1", inplace=True)
df['Cabin_class'] = df['Cabin'].str.get(0)

In [258]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [259]:
df_temp =  pd.get_dummies(df[['Pclass', 'title', 'Sex', 'Embarked', 'Cabin_class']])

In [260]:
df2 = pd.concat([df, df_temp], axis=1)
df2 = df2.drop(['Pclass', 'title', 'Sex', 'Embarked', 'Cabin_class'], axis=1)

In [261]:
train_data = df2[:df_train.shape[0]]
test_data = df2[df_train.shape[0]:]

In [262]:
train_data = train_data.drop(['PassengerId'], axis=1)
test_id = test_data[['PassengerId']]
test_data = test_data.drop(['PassengerId'], axis=1)

In [263]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train_data, train_Y, test_size=0.25, random_state=4)

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 4)
clf = RandomForestClassifier(n_estimators=20, max_depth=4)

# 訓練模型
clf.fit(x_train, y_train)

# 預測測試集
y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.8385650224215246


In [264]:
clf = RandomForestClassifier(n_estimators=20, max_depth=4)
clf.fit(train_data, train_Y)
y_pred = clf.predict(test_data)

In [266]:
test_id['lable'] = list(y_pred)

In [268]:
test_id.to_csv('base1.csv')